In [1]:
from Simulation.mpc import *
from Simulation.system_functions import PolymerCSTR
from utils.helpers import *

## Initialize the system

In [2]:
# First initiate the system
# Parameters
Ad = 2.142e17           # h^-1
Ed = 14897              # K
Ap = 3.816e10           # L/(molh)
Ep = 3557               # K
At = 4.50e12            # L/(molh)
Et = 843                # K
fi = 0.6                # Coefficient
m_delta_H_r = -6.99e4   # j/mol
hA = 1.05e6             # j/(Kh)
rhocp = 1506            # j/(Kh)
rhoccpc = 4043          # j/(Kh)
Mm = 104.14             # g/mol
system_params = np.array([Ad, Ed, Ap, Ep, At, Et, fi, m_delta_H_r, hA, rhocp, rhoccpc, Mm])

In [3]:
# Design Parameters
CIf = 0.5888    # mol/L
CMf = 8.6981    # mol/L
Qi = 108.       # L/h
Qs = 459.       # L/h
Tf = 330.       # K
Tcf = 295.      # K
V = 3000.       # L
Vc = 3312.4     # L
        
system_design_params = np.array([CIf, CMf, Qi, Qs, Tf, Tcf, V, Vc])

In [4]:
# Steady State Inputs
Qm_ss = 378.    # L/h
Qc_ss = 471.6   # L/h

system_steady_state_inputs = np.array([Qc_ss, Qm_ss])

In [5]:
# Sampling time of the system
delta_t = 0.5 # 30 mins

In [6]:
# Initiate the CSTR for steady state values
cstr = PolymerCSTR(system_params, system_design_params, system_steady_state_inputs, delta_t)
steady_states={"ss_inputs":cstr.ss_inputs,
               "y_ss":cstr.y_ss}

## Loading the system matrices, min max scaling, and min max of the states

In [7]:
dir_path = os.path.join(os.getcwd(), "Data")

In [8]:
# Defining the range of setpoints for data generation
setpoint_y = np.array([[3.2, 321],
                       [4.5, 325]])
u_min = np.array([71.6, 78])
u_max = np.array([870, 670])

system_data = load_and_prepare_system_data(steady_states=steady_states, setpoint_y=setpoint_y, u_min=u_min, u_max=u_max)

In [9]:
A_aug = system_data["A_aug"]
B_aug = system_data["B_aug"]
C_aug = system_data["C_aug"]

In [10]:
data_min = system_data["data_min"]
data_max = system_data["data_max"]

In [11]:
min_max_states = {'max_s': np.array([256.79686253, 256.01560603,  48.99447186, 144.79949103,
          2.82199733,   3.14014989,   2.78866348,   3.71691422,
          6.2029936 ]),
                  'min_s': np.array([ -272.28060121, -1112.33972595,   -76.63993491,  -608.60327886,
           -3.94399122,    -3.93115257,    -2.9532091 ,    -4.06547624,
          -28.25906582])}

In [12]:
y_sp_scaled_deviation = system_data["y_sp_scaled_deviation"]

In [13]:
b_min = system_data["b_min"]
b_max = system_data["b_max"]

In [14]:
min_max_dict = system_data["min_max_dict"]
min_max_dict["x_max"] = np.array([256.79686253, 256.01560603,  48.99447186, 144.79949103,
          2.82199733,   3.14014989,   2.78866348,   3.71691422,
          6.2029936 ])
min_max_dict["x_min"] = np.array([ -272.28060121, -1112.33972595,   -76.63993491,  -608.60327886,
           -3.94399122,    -3.93115257,    -2.9532091 ,    -4.06547624,
          -28.25906582])

In [15]:
# Setpoints in deviation form
inputs_number = int(B_aug.shape[1])
y_sp_scenario = np.array([[4.5, 324],
                          [3.4, 321]])

y_sp_scenario = (apply_min_max(y_sp_scenario, data_min[inputs_number:], data_max[inputs_number:])
                 - apply_min_max(steady_states["y_ss"], data_min[inputs_number:], data_max[inputs_number:]))
n_tests = 200
set_points_len = 400
TEST_CYCLE = [False, False, False, False, False]
warm_start = 10
ACTOR_FREEZE = 10 * set_points_len
warm_start_plot = warm_start * 2 * set_points_len + ACTOR_FREEZE

In [16]:
# Observer Gain
poles = np.array(np.array([0.44619852, 0.33547649, 0.36380595, 0.70467118, 0.3562966,
                           0.42900673, 0.4228262 , 0.96916776, 0.91230187]))
L = compute_observer_gain(A_aug, C_aug, poles)

The system is observable.


C:\Users\HAMEDI\OneDrive - McMaster University\PythonProjects\Polymer_example\Simulation\mpc.py:124: UserWarning: Convergence was not reached after maxiter iterations.
You asked for a tolerance of 0.001, we got 0.9999999422182038.
  obs_gain_calc = signal.place_poles(A.T, C.T, desired_poles, method='KNV0')


## Setting The hyperparameters for the TD3 Agent

In [17]:
from TD3Agent.agent import TD3Agent
import torch

In [18]:
set_points_number = int(C_aug.shape[0])
inputs_number = int(B_aug.shape[1])
STATE_DIM = int(A_aug.shape[0]) + set_points_number + inputs_number
ACTION_DIM = int(B_aug.shape[1])
n_outputs = C_aug.shape[0]
ACTOR_LAYER_SIZES = [512, 512, 512, 512, 512]
CRITIC_LAYER_SIZES = [512, 512, 512, 512, 512]
BUFFER_CAPACITY = 40000
ACTOR_LR = 5e-5
CRITIC_LR = 5e-4
SMOOTHING_STD = 0.005
NOISE_CLIP = 0.01
# EXPLORATION_NOISE_STD = 0.01
GAMMA = 0.995
TAU = 0.005 # 0.01
MAX_ACTION = 1
POLICY_DELAY = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 256
STD_START = 0.02
STD_END = 0.001
STD_DECAY_RATE = 0.99992
STD_DECAY_MODE = "exp"

In [19]:
td3_agent = TD3Agent(
    state_dim=STATE_DIM,
    action_dim=ACTION_DIM,
    actor_hidden=ACTOR_LAYER_SIZES,
    critic_hidden=CRITIC_LAYER_SIZES,
    gamma=GAMMA,
    actor_lr=ACTOR_LR,
    critic_lr=CRITIC_LR,
    batch_size=BATCH_SIZE,
    policy_delay=POLICY_DELAY,
    target_policy_smoothing_noise_std=SMOOTHING_STD,
    noise_clip=NOISE_CLIP,
    max_action=MAX_ACTION,
    tau=TAU,
    std_start=STD_START,
    std_end=STD_END,
    std_decay_rate=STD_DECAY_RATE,
    std_decay_mode=STD_DECAY_MODE,
    buffer_size=BUFFER_CAPACITY,
    device=DEVICE,
    actor_freeze=ACTOR_FREEZE,
    )

In [20]:
agent_path = r"C:\Users\HAMEDI\OneDrive - McMaster University\PythonProjects\Polymer_example\Data\models\agent_2507171027.pkl"
td3_agent.load(agent_path)

Agent loaded successfully from: C:\Users\HAMEDI\OneDrive - McMaster University\PythonProjects\Polymer_example\Data\models\agent_2507171027.pkl


# MPC Initialization

In [21]:
# MPC parameters
predict_h = 9
cont_h = 3
b1 = (b_min[0], b_max[0])
b2 = (b_min[1], b_max[1])
bnds = (b1, b2)*cont_h
cons = []
IC_opt = np.zeros(inputs_number*cont_h)
Q1_penalty = 5.
Q2_penalty = 1.
R1_penalty = 1.
R2_penalty = 1.
Q_penalty = np.array([[Q1_penalty, 0], [0, Q2_penalty]])
R_penalty = np.array([[R1_penalty, 0], [0, R2_penalty]])

In [22]:
MPC_obj = MpcSolver(A_aug, B_aug, C_aug,
                    Q1_penalty, Q2_penalty, R1_penalty, R2_penalty,
                    predict_h, cont_h)

## Applying RL Agent on the CSTR

In [23]:
def make_reward_fn_relative_QR(
    data_min, data_max, n_inputs,
    k_rel, band_floor_phys,
    Q_diag, R_diag,
    tau_frac=0.7,
    gamma_out=0.5, gamma_in=0.5,
    beta=5.0, gate="geom", lam_in=1.0,
    bonus_kind="exp", bonus_k=12.0, bonus_p=0.6, bonus_c=20.0,
):
    """
    Reward with relative tracking bands.

    data_min, data_max : arrays for [u_min..., y_min...], [u_max..., y_max...]
    n_inputs           : number of inputs (so outputs start at index n_inputs)
    k_rel              : per-output relative tolerance factors (same length as outputs)
    band_floor_phys    : per-output minimum band in physical units
    Q_diag, R_diag     : quadratic weights (same as before)
    """

    data_min = np.asarray(data_min, float)
    data_max = np.asarray(data_max, float)
    dy = np.maximum(data_max[n_inputs:] - data_min[n_inputs:], 1e-12)  # phys range for each y

    k_rel = np.asarray(k_rel, float)
    band_floor_phys = np.asarray(band_floor_phys, float)
    Q_diag = np.asarray(Q_diag, float)
    R_diag = np.asarray(R_diag, float)

    # floor in *scaled* coordinates (used if y_sp_phys is not provided)
    band_floor_scaled = band_floor_phys / np.maximum(dy, 1e-12)

    def _sigmoid(x):
        x = np.clip(x, -60.0, 60.0)
        return 1.0 / (1.0 + np.exp(-x))

    def _phi(z, kind=bonus_kind, k=bonus_k, p=bonus_p, c=bonus_c):
        z = np.clip(z, 0.0, 1.0)
        if kind == "linear":
            return 1.0 - z
        if kind == "quadratic":
            return (1.0 - z) ** 2
        if kind == "exp":
            return (np.exp(-k * z) - np.exp(-k)) / (1.0 - np.exp(-k))
        if kind == "power":
            return 1.0 - np.power(z, p)
        if kind == "log":
            return np.log1p(c * (1.0 - z)) / np.log1p(c)
        raise ValueError("unknown bonus kind")

    def reward_fn(e_scaled, du_scaled, y_sp_phys=None):
        """
        e_scaled : output error in scaled deviation space  (same as before)
        du_scaled: input move in scaled deviation space    (same as before)
        y_sp_phys: current setpoint in *physical* units (array len = n_outputs)
        """

        e_scaled = np.asarray(e_scaled, float)
        du_scaled = np.asarray(du_scaled, float)

        # ----- dynamic band based on setpoint -----
        if y_sp_phys is None:
            # fallback: just use the floor
            band_scaled = band_floor_scaled
        else:
            y_sp_phys_arr = np.asarray(y_sp_phys, float)
            # band_phys_i = max(k_rel_i * |y_sp_i|, band_floor_phys_i)
            band_phys = np.maximum(k_rel * np.abs(y_sp_phys_arr), band_floor_phys)
            band_scaled = band_phys / np.maximum(dy, 1e-12)

        tau_scaled = tau_frac * band_scaled

        # ----- inside/outside gate -----
        abs_e = np.abs(e_scaled)
        s_i = _sigmoid((band_scaled - abs_e) / np.maximum(tau_scaled, 1e-12))

        if gate == "prod":
            w_in = float(np.prod(s_i, dtype=np.float64))
        elif gate == "mean":
            w_in = float(np.mean(s_i))
        elif gate == "geom":
            w_in = float(np.prod(s_i, dtype=np.float64) ** (1.0 / len(s_i)))
        else:
            raise ValueError("gate must be 'prod'|'mean'|'geom'")

        # ----- core quadratic costs -----
        err_quad = np.sum(Q_diag * (e_scaled ** 2))
        err_eff = (1.0 - w_in) * err_quad + w_in * (lam_in * err_quad)
        move = np.sum(R_diag * (du_scaled ** 2))

        # ----- linear penalties around band edge -----
        slope_at_edge = 2.0 * Q_diag * band_scaled

        overflow = np.maximum(abs_e - band_scaled, 0.0)
        lin_out = (1.0 - w_in) * np.sum(gamma_out * slope_at_edge * overflow)

        inside_mag = np.minimum(abs_e, band_scaled)
        lin_in = w_in * np.sum(gamma_in * slope_at_edge * inside_mag)

        # ----- bonus near zero error -----
        qb2 = Q_diag * (band_scaled ** 2)
        z = abs_e / np.maximum(band_scaled, 1e-12)
        phi = _phi(z)
        bonus = w_in * beta * np.sum(qb2 * phi)

        # ----- total reward -----
        return -(err_eff + move + lin_out + lin_in) + bonus

    params = dict(
        k_rel=k_rel,
        band_floor_phys=band_floor_phys,
        band_floor_scaled=band_floor_scaled,
        Q_diag=Q_diag,
        R_diag=R_diag,
        tau_frac=tau_frac,
        gamma_out=gamma_out,
        gamma_in=gamma_in,
        beta=beta,
        gate=gate,
        lam_in=lam_in,
        bonus_kind=bonus_kind,
        bonus_k=bonus_k,
        bonus_p=bonus_p,
        bonus_c=bonus_c,
    )
    return params, reward_fn

## Reward configuration

In [24]:
n_inputs = 2

dy = data_max[n_inputs:] - data_min[n_inputs:]
y_sp_nom = 0.5 * (data_min[n_inputs:] + data_max[n_inputs:])

k_rel = np.array([0.003, 0.0003])
band_floor_phys = np.array([0.006, 0.07])

band_phys = np.maximum(k_rel * np.abs(y_sp_nom), band_floor_phys)

scale_factor = 1.0  # use 2.0 for [-1, 1] scaling, 1.0 for [0, 1]
band_scaled = scale_factor * band_phys / dy

q0 = 1.4
Q_diag = q0 / np.maximum(band_scaled ** 2, 1e-12)

print("dy:", dy)
print("y_sp_nom:", y_sp_nom)
print("band_phys:", band_phys)
print("band_scaled:", band_scaled)
print("Q_diag:", Q_diag)

dy: [0.22165278 0.78153727]
y_sp_nom: [  3.83915067 323.21371982]
band_phys: [0.01151745 0.09696412]
band_scaled: [0.05196169 0.12406845]
Q_diag: [518.51529284  90.95055189]


In [25]:
Q_diag = np.array([518., 90.])          # rounded from the band-based calculation
R_diag = np.array([90., 90.])          # move cost for du_scaled ~ 0.02

n_inputs = 2

print("Band scaled are:")

params, reward_fn = make_reward_fn_relative_QR(
    data_min, data_max, n_inputs,
    k_rel, band_floor_phys,
    Q_diag, R_diag,
    tau_frac=0.7,
    gamma_out=0.5, gamma_in=0.5,
    beta=7.0, gate="geom", lam_in=1.0,
    bonus_kind="exp", bonus_k=12.0, bonus_p=0.6, bonus_c=20.0,
)
print(params)

Band scaled are:
{'k_rel': array([0.003 , 0.0003]), 'band_floor_phys': array([0.006, 0.07 ]), 'band_floor_scaled': array([0.02706937, 0.08956707]), 'Q_diag': array([518.,  90.]), 'R_diag': array([90., 90.]), 'tau_frac': 0.7, 'gamma_out': 0.5, 'gamma_in': 0.5, 'beta': 7.0, 'gate': 'geom', 'lam_in': 1.0, 'bonus_kind': 'exp', 'bonus_k': 12.0, 'bonus_p': 0.6, 'bonus_c': 20.0}


In [26]:
nominal_qs = 459
nominal_qi = 108
nominal_hA = 1.05e6
qi_change = 0.95
qs_change = 1.05
ha_change = 0.92

In [27]:
def run_rl_train(system, y_sp_scenario, n_tests, set_points_len,
                 steady_states, min_max_dict, agent, MPC_obj,
                 L, data_min, data_max, warm_start,
                 test_cycle,
                 nominal_qi, nominal_qs, nominal_ha,
                 qi_change, qs_change, ha_change,
                 reward_fn, mode="disturb"):

    # --- setpoints generation ---
    y_sp, nFE, sub_episodes_changes_dict, time_in_sub_episodes, test_train_dict, WARM_START, qi, qs, ha = \
        generate_setpoints_training_rl_gradually(
            y_sp_scenario, n_tests, set_points_len, warm_start, test_cycle,
            nominal_qi, nominal_qs, nominal_ha,
            qi_change, qs_change, ha_change
        )

    # inputs and outputs of the system dimensions
    n_inputs = B_aug.shape[1]
    n_outputs = C_aug.shape[0]
    n_states = A_aug.shape[0]

    # Scaled steady states inputs and outputs
    ss_scaled_inputs = apply_min_max(steady_states["ss_inputs"], data_min[:n_inputs], data_max[:n_inputs])
    y_ss_scaled = apply_min_max(steady_states["y_ss"], data_min[n_inputs:], data_max[n_inputs:])
    u_min, u_max = min_max_dict["u_min"], min_max_dict["u_max"]

    y_system = np.zeros((nFE + 1, n_outputs))
    y_system[0, :] = system.current_output
    u_rl = np.zeros((nFE, n_inputs))
    yhat = np.zeros((n_outputs, nFE))
    xhatdhat = np.zeros((n_states, nFE + 1))
    # xhatdhat[:, 0] = np.random.uniform(low=min_max_dict["x_min"], high=min_max_dict["x_max"])
    rewards = np.zeros(nFE)
    avg_rewards = []

    delta_y_storage = []

    # ----- helper ------
    def map_to_bounds(a, low, high):
        return low + ((a + 1.0) / 2.0) * (high - low)

    test = False

    for i in range(nFE):
        # train/test phase
        if i in test_train_dict:
            test = test_train_dict[i]

        # Current scaled input & deviation
        scaled_current_input = apply_min_max(system.current_input, data_min[:n_inputs], data_max[:n_inputs])
        scaled_current_input_dev = scaled_current_input - ss_scaled_inputs

        # ---- RL state (scaled) ----
        current_rl_state = apply_rl_scaled(min_max_dict, xhatdhat[:, i], y_sp[i, :], scaled_current_input_dev)

        # ---- TD3 action ----
        if not test:
            action = agent.take_action(current_rl_state, explore=(not test))
        else:
            action = agent.act_eval(current_rl_state)
        # Map to bounds
        u_scaled = map_to_bounds(action, u_min, u_max)

        # scale & step plant
        u_rl[i, :] = u_scaled + ss_scaled_inputs
        u_plant = reverse_min_max(u_rl[i, :], data_min[:n_inputs], data_max[:n_inputs])

        # delta u cost variables
        delta_u = u_rl[i, :] - scaled_current_input

        # Apply to plant and step
        system.current_input = u_plant
        system.step()
        if mode == "disturb":
            # disturbances
            system.hA = ha[i]
            system.Qs = qs[i]
            system.Qi = qi[i]

        # Record output
        y_system[i+1, :] = system.current_output

        # ----- Observer & model roll -----
        y_current_scaled = apply_min_max(y_system[i+1, :], data_min[n_inputs:], data_max[n_inputs:]) - y_ss_scaled
        y_prev_scaled = apply_min_max(y_system[i, :], data_min[n_inputs:], data_max[n_inputs:]) - y_ss_scaled

        # Calculate Delta y in deviation form
        delta_y = y_current_scaled - y_sp[i, :]

        # Calculate the next state in deviation form
        yhat[:, i] = np.dot(MPC_obj.C, xhatdhat[:, i])
        xhatdhat[:, i+1] = np.dot(MPC_obj.A, xhatdhat[:, i]) + np.dot(MPC_obj.B, (u_rl[i, :] - ss_scaled_inputs)) + np.dot(L, (y_prev_scaled - yhat[:, i])).T

        # y_sp in physical band
        y_sp_phys = reverse_min_max(y_sp[i, :] + y_ss_scaled, data_min[n_inputs:], data_max[n_inputs:])

        # Reward Calculation
        reward = reward_fn(delta_y, delta_u, y_sp_phys)

        # Record rewards and delta_y
        rewards[i] = reward * 0.01
        delta_y_storage.append(np.abs(delta_y))

        # ----- Next state for TD3 -----
        next_u_dev = u_rl[i, :] - ss_scaled_inputs
        next_rl_state = apply_rl_scaled(min_max_dict, xhatdhat[:, i+1], y_sp[i, :], next_u_dev)

        # Episode boundary (treat each setpoint block as an episode end)
        # done = 1.0 if (i + 1) % boundary == 0 else 0.0
        done = 0.0

        # Buffer + train (skip if in test phase)
        if not test:
            agent.push(current_rl_state,
                       action.astype(np.float32),
                       float(reward),
                       next_rl_state,
                       float(done))
            if i >= WARM_START:
                _ = agent.train_step()  # returns loss or None

        # diagnostics at sub-episode boundary
        if i in sub_episodes_changes_dict:
            avg_rewards.append(np.mean(rewards[max(0, i - time_in_sub_episodes + 1): i + 1]))
            print('Sub_Episode:', sub_episodes_changes_dict[i], '| avg. reward:', avg_rewards[-1])
            if hasattr(agent, "_expl_sigma"):
                print('Exploration noise:', agent._expl_sigma)

    # unscale to plant units for plotting
    u_rl = reverse_min_max(u_rl, data_min[:n_inputs], data_max[:n_inputs])

    return y_system, u_rl, avg_rewards, rewards, xhatdhat, nFE, time_in_sub_episodes, y_sp, yhat, delta_y_storage, qi, qs, ha

In [28]:
cstr = PolymerCSTR(system_params, system_design_params, system_steady_state_inputs, delta_t)
y_system, u_rl, avg_rewards, rewards, xhatdhat, nFE, time_in_sub_episodes, y_sp, yhat, delta_y_storage, qi, qs, ha\
    = run_rl_train(cstr, y_sp_scenario, n_tests, set_points_len,
                 steady_states, min_max_dict, td3_agent, MPC_obj,
                 L, data_min, data_max, warm_start,
                 TEST_CYCLE,
                 nominal_qi, nominal_qs, nominal_hA,
                 qi_change, qs_change, ha_change,
                 reward_fn)

Sub_Episode: 1 | avg. reward: -20.915602920324947
Exploration noise: 0.018822049364146075
Sub_Episode: 2 | avg. reward: -26.447110867549572
Exploration noise: 0.017717128607266288
Sub_Episode: 3 | avg. reward: -20.707880590916194
Exploration noise: 0.016680710066604123
Sub_Episode: 4 | avg. reward: -23.019523546931467
Exploration noise: 0.01570854678273058
Sub_Episode: 5 | avg. reward: -22.10909622675608
Exploration noise: 0.014796655096667173
Sub_Episode: 6 | avg. reward: -25.37481006259389
Exploration noise: 0.013941298325942101
Sub_Episode: 7 | avg. reward: -24.26070243454449
Exploration noise: 0.013138971452688476
Sub_Episode: 8 | avg. reward: -24.105362708554836
Exploration noise: 0.01238638676104074
Sub_Episode: 9 | avg. reward: -23.003397936185756
Exploration noise: 0.011680460364975123
Sub_Episode: 10 | avg. reward: -23.80032570459916
Exploration noise: 0.011018299571389067
Sub_Episode: 11 | avg. reward: -24.87914175136513
Exploration noise: 0.010397191026636811
Sub_Episode: 12

In [29]:
def plot_rl_results_disturbance(
    y_sp, steady_states, nFE, delta_t, time_in_sub_episodes,
    y_mpc, u_mpc, avg_rewards, data_min, data_max, warm_start_plot,
    directory=None, prefix_name="agent_result",
    agent=None,
    delta_y_storage=None,
    rewards=None,
    dist=None,
    start_plot_idx=10
):
    """
    Distillation-style plotting (same colors/fonts/no legends).
    Saves all figures + input_data.pkl to directory/prefix_name/<timestamp>.
    Handles:
      dist=None
      dist=1D array
      dist=dict with keys {"qi","qs","ha"}
    """
    import os
    import pickle
    from datetime import datetime

    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    import matplotlib.ticker as mtick

    from utils.helpers import apply_min_max, reverse_min_max

    if directory is None:
        directory = os.getcwd()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_dir = os.path.join(directory, prefix_name, timestamp)
    os.makedirs(out_dir, exist_ok=True)

    def _savefig(name):
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, name), bbox_inches="tight", dpi=300)
        plt.close()

    y_sp_original = np.array(y_sp, copy=True)

    actor_losses = getattr(agent, "actor_losses", None) if agent is not None else None
    critic_losses = getattr(agent, "critic_losses", None) if agent is not None else None
    dy_arr = np.array(delta_y_storage) if delta_y_storage is not None else None
    rewards_arr = np.array(rewards) if rewards is not None else None

    input_data = {
        "y_sp": y_sp_original,
        "steady_states": steady_states,
        "nFE": nFE,
        "delta_t": delta_t,
        "time_in_sub_episodes": time_in_sub_episodes,
        "y_mpc": y_mpc,
        "u_mpc": u_mpc,
        "avg_rewards": avg_rewards,
        "data_min": data_min,
        "data_max": data_max,
        "warm_start_plot": warm_start_plot,
        "actor_losses": actor_losses,
        "critic_losses": critic_losses,
        "delta_y_storage": dy_arr,
        "rewards": rewards_arr,
        "dist": dist,
        "start_plot_idx": start_plot_idx
    }
    with open(os.path.join(out_dir, "input_data.pkl"), "wb") as f:
        pickle.dump(input_data, f)

    # Canceling the deviation form (same logic)
    y_ss = apply_min_max(steady_states["y_ss"], data_min[2:], data_max[2:])
    y_sp = (y_sp + y_ss)
    y_sp = (reverse_min_max(y_sp, data_min[2:], data_max[2:])).T  # (n_out, nFE)

    # Distillation-style rcParams (no bold globals; bold comes from \mathbf in labels)
    mpl.rcParams.update({
        "font.size": 12,
        "axes.grid": True,
        "grid.linestyle": "--",
        "grid.linewidth": 0.6,
        "grid.alpha": 0.35,
        "legend.frameon": True
    })

    # Colors exactly like distillation code
    C_QC = "tab:green"
    C_QM = "tab:orange"
    C_RW = "tab:purple"

    time_plot = np.linspace(0, nFE * delta_t, nFE + 1)
    warm_start_plot = np.atleast_1d(warm_start_plot) * delta_t
    ws_end = float(warm_start_plot.max()) if warm_start_plot.size > 0 else 0.0

    time_plot_hour = np.linspace(0, time_in_sub_episodes * delta_t, time_in_sub_episodes + 1)

    # -------- Plot 1: outputs (full) --------
    plt.figure(figsize=(10, 8))

    ax = plt.subplot(2, 1, 1)
    ax.plot(time_plot[start_plot_idx:], y_mpc[start_plot_idx:, 0], "b-", lw=2, zorder=2)
    ax.step(time_plot[start_plot_idx:-1], y_sp[0, start_plot_idx:], "r--", lw=2, where="post", zorder=3)
    for t_ws in warm_start_plot:
        ax.axvline(float(t_ws), color="k", linestyle="--", linewidth=1.2, zorder=1)
    if ws_end > 0.0:
        ax.axvspan(0.0, ws_end, facecolor="0.9", alpha=0.6, zorder=0)
    ax.set_ylabel(r"$\mathbf{\eta}$ (L/g)", fontsize=18)
    ax.set_xlim(0, time_plot[-1])
    ax.xaxis.set_major_locator(mtick.MaxNLocator(6))
    ax.xaxis.set_minor_locator(mtick.AutoMinorLocator(2))
    ax.xaxis.set_major_formatter(mtick.FormatStrFormatter("%d"))
    ax.tick_params(axis="x", pad=4)

    ax = plt.subplot(2, 1, 2)
    ax.plot(time_plot[start_plot_idx:], y_mpc[start_plot_idx:, 1], "b-", lw=2, zorder=2)
    ax.step(time_plot[start_plot_idx:-1], y_sp[1, start_plot_idx:], "r--", lw=2, where="post", zorder=3)
    for t_ws in warm_start_plot:
        ax.axvline(float(t_ws), color="k", linestyle="--", linewidth=1.2, zorder=1)
    if ws_end > 0.0:
        ax.axvspan(0.0, ws_end, facecolor="0.9", alpha=0.6, zorder=0)
    ax.set_ylabel(r"$\mathbf{T}$ (K)", fontsize=18)
    ax.set_xlabel(r"$\mathbf{Time}$ (hour)", fontsize=18)
    ax.set_xlim(0, time_plot[-1])
    ax.xaxis.set_major_locator(mtick.MaxNLocator(6))
    ax.xaxis.set_minor_locator(mtick.AutoMinorLocator(2))
    ax.xaxis.set_major_formatter(mtick.FormatStrFormatter("%d"))
    ax.tick_params(axis="x", pad=4)

    plt.subplot(2, 1, 1)
    plt.tick_params(axis="both", labelsize=16)
    plt.subplot(2, 1, 2)
    plt.tick_params(axis="both", labelsize=16)

    plt.gcf().subplots_adjust(right=0.95, bottom=0.12)
    _savefig("fig_rl_outputs_full.png")

    # -------- last window --------
    plt.figure(figsize=(7.6, 5.2))

    ax = plt.subplot(2, 1, 1)
    ax.plot(time_plot_hour, y_mpc[nFE - time_in_sub_episodes:, 0], "-", lw=2.2, color="b", zorder=2)
    ax.step(time_plot_hour[:-1], y_sp[0, nFE - time_in_sub_episodes:], where="post",
            linestyle="--", lw=2.2, color="r", alpha=0.95, zorder=3)
    ax.set_ylabel(r"$\eta$ (L/g)")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax = plt.subplot(2, 1, 2)
    ax.plot(time_plot_hour, y_mpc[nFE - time_in_sub_episodes:, 1], "-", lw=2.2, color="b", zorder=2)
    ax.step(time_plot_hour[:-1], y_sp[1, nFE - time_in_sub_episodes:], where="post",
            linestyle="--", lw=2.2, color="r", alpha=0.95, zorder=3)
    ax.set_ylabel(r"$T$ (K)")
    ax.set_xlabel("Time (h)")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.gcf().subplots_adjust(right=0.95)
    _savefig(f"fig_rl_outputs_last{time_in_sub_episodes}.png")

    # -------- last 4x window --------
    W4 = 4 * time_in_sub_episodes
    time_plot_4w = np.linspace(0, W4 * delta_t, W4 + 1)

    plt.figure(figsize=(7.6, 5.2))

    ax = plt.subplot(2, 1, 1)
    ax.plot(time_plot_4w, y_mpc[nFE - W4:, 0], "-", lw=2.2, color="b", zorder=2)
    ax.step(time_plot_4w[:-1], y_sp[0, nFE - W4:], where="post",
            linestyle="--", lw=2.2, color="r", alpha=0.95, zorder=3)
    ax.set_ylabel(r"$\eta$ (L/g)")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax = plt.subplot(2, 1, 2)
    ax.plot(time_plot_4w, y_mpc[nFE - W4:, 1], "-", lw=2.2, color="b", zorder=2)
    ax.step(time_plot_4w[:-1], y_sp[1, nFE - W4:], where="post",
            linestyle="--", lw=2.2, color="r", alpha=0.95, zorder=3)
    ax.set_ylabel(r"$T$ (K)")
    ax.set_xlabel("Time (h)")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.gcf().subplots_adjust(right=0.95)
    _savefig(f"fig_rl_outputs_last{W4}.png")

    # -------- Plot 2: inputs --------
    plt.figure(figsize=(7.6, 5.2))

    ax = plt.subplot(2, 1, 1)
    ax.step(time_plot[:-1], u_mpc[:, 0], where="post", lw=2.2, color=C_QC, zorder=2)
    ax.set_ylabel(r"$Q_c$ (L/h)")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax = plt.subplot(2, 1, 2)
    ax.step(time_plot[:-1], u_mpc[:, 1], where="post", lw=2.2, color=C_QM, zorder=2)
    ax.set_ylabel(r"$Q_m$ (L/h)")
    ax.set_xlabel("Time (h)")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.gcf().subplots_adjust(right=0.95)
    _savefig("fig_rl_inputs_full.png")

    # -------- Plot 3: reward per episode --------
    plt.figure(figsize=(7.2, 4.2))
    xep = np.arange(1, len(avg_rewards) + 1)
    plt.plot(xep, avg_rewards, "o-", lw=2.2, color=C_RW, zorder=2)
    plt.ylabel("Avg. Reward")
    plt.xlabel("Episode #")
    plt.grid(True, which="both", linestyle="--", linewidth=0.6, alpha=0.35)
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    _savefig("fig_rl_rewards.png")

    # -------- optional losses --------
    if actor_losses is not None and len(actor_losses) > 0:
        plt.figure(figsize=(7.2, 4.2))
        plt.plot(actor_losses, lw=1.8, color="tab:blue")
        plt.ylabel("Actor Loss")
        plt.xlabel("Update Step")
        plt.grid(True, linestyle="--", alpha=0.35)
        _savefig("loss_actor.png")

    if critic_losses is not None and len(critic_losses) > 0:
        plt.figure(figsize=(7.2, 4.2))
        plt.plot(critic_losses, lw=1.8, color="tab:orange")
        plt.ylabel("Critic Loss")
        plt.xlabel("Update Step")
        plt.grid(True, linestyle="--", alpha=0.35)
        _savefig("loss_critic.png")

    # -------- optional delta_y windows (no legend) --------
    if dy_arr is not None and dy_arr.ndim == 2 and dy_arr.shape[1] >= 2:
        n = dy_arr.shape[0]

        i0 = max(0, n - 300)
        w = dy_arr[i0:n]
        if len(w) > 0:
            plt.figure(figsize=(7.6, 4.2))
            plt.plot(w[:, 0], c="r")
            plt.plot(w[:, 1], c="b")
            plt.ylabel(r"$\Delta y$")
            plt.xlabel("Step")
            plt.grid(True, linestyle="--", alpha=0.35)
            _savefig("delta_y_last300.png")

        j0 = max(0, n - 700)
        j1 = max(0, n - 400)
        w2 = dy_arr[j0:j1]
        if len(w2) > 0:
            plt.figure(figsize=(7.6, 4.2))
            plt.plot(w2[:, 0], c="r")
            plt.plot(w2[:, 1], c="b")
            plt.ylabel(r"$\Delta y$")
            plt.xlabel("Step")
            plt.grid(True, linestyle="--", alpha=0.35)
            _savefig("delta_y_700_400.png")

    # -------- optional per-step rewards (no legend) --------
    if rewards_arr is not None and rewards_arr.ndim == 1 and rewards_arr.size > 0:
        n = rewards_arr.size

        j0 = max(0, n - 700)
        j1 = max(0, n - 400)
        w = rewards_arr[j0:j1]
        if w.size > 0:
            plt.figure(figsize=(7.6, 4.2))
            plt.scatter(range(w.size), w, s=10)
            plt.ylabel("Reward")
            plt.xlabel("Step")
            plt.grid(True, linestyle="--", alpha=0.35)
            _savefig("rewards_700_400.png")

        i0 = max(0, n - 300)
        w2 = rewards_arr[i0:n]
        if w2.size > 0:
            plt.figure(figsize=(7.6, 4.2))
            plt.scatter(range(w2.size), w2, s=10)
            plt.ylabel("Reward")
            plt.xlabel("Step")
            plt.grid(True, linestyle="--", alpha=0.35)
            _savefig("rewards_last300.png")

        plt.figure(figsize=(7.6, 4.2))
        plt.scatter(range(rewards_arr.size), rewards_arr, s=10)
        plt.ylabel("Reward")
        plt.xlabel("Step")
        plt.grid(True, linestyle="--", alpha=0.35)
        _savefig("rewards_all.png")

    # -------- disturbance (no legend) --------
    if dist is not None:
        if isinstance(dist, dict) and all(k in dist for k in ["qi", "qs", "ha"]):
            qi_arr = np.asarray(dist["qi"]).squeeze()
            qs_arr = np.asarray(dist["qs"]).squeeze()
            ha_arr = np.asarray(dist["ha"]).squeeze()
            n_al = min(nFE, qi_arr.shape[0], qs_arr.shape[0], ha_arr.shape[0])

            def _dist_fig(t, q1, q2, hA, suffix):
                plt.figure(figsize=(7.6, 6.2))

                ax = plt.subplot(3, 1, 1)
                ax.plot(t, q1, "-", lw=2, color="tab:blue")
                ax.set_ylabel(r"$Q_i$ (L/h)")
                ax.spines["top"].set_visible(False)
                ax.spines["right"].set_visible(False)
                ax.xaxis.set_major_locator(mtick.MaxNLocator(6))
                ax.xaxis.set_minor_locator(mtick.AutoMinorLocator(2))
                ax.grid(True, linestyle="--", alpha=0.35)

                ax = plt.subplot(3, 1, 2)
                ax.plot(t, q2, "-", lw=2, color="tab:orange")
                ax.set_ylabel(r"$Q_s$ (L/h)")
                ax.spines["top"].set_visible(False)
                ax.spines["right"].set_visible(False)
                ax.xaxis.set_major_locator(mtick.MaxNLocator(6))
                ax.xaxis.set_minor_locator(mtick.AutoMinorLocator(2))
                ax.grid(True, linestyle="--", alpha=0.35)

                ax = plt.subplot(3, 1, 3)
                ax.plot(t, hA, "-", lw=2, color="tab:green")
                ax.set_xlabel("Time (h)")
                ax.set_ylabel(r"$h_a$ (J/Kh)")
                ax.spines["top"].set_visible(False)
                ax.spines["right"].set_visible(False)
                ax.xaxis.set_major_locator(mtick.MaxNLocator(6))
                ax.xaxis.set_minor_locator(mtick.AutoMinorLocator(2))
                ax.grid(True, linestyle="--", alpha=0.35)

                plt.gcf().subplots_adjust(right=0.95, hspace=0.25)
                _savefig(f"fig_disturbances_{suffix}.png")

            _dist_fig(time_plot[:n_al], qi_arr[:n_al], qs_arr[:n_al], ha_arr[:n_al], suffix="full")

            if time_in_sub_episodes > 0:
                W = min(time_in_sub_episodes, n_al)
                t_lastW = np.linspace(0, W * delta_t, W, endpoint=False)
                _dist_fig(
                    t_lastW,
                    qi_arr[n_al - W:n_al],
                    qs_arr[n_al - W:n_al],
                    ha_arr[n_al - W:n_al],
                    suffix=f"last{W}"
                )
        else:
            dist_arr = np.asarray(dist).squeeze()
            n_al = min(nFE, dist_arr.shape[0])
            plt.figure(figsize=(7.2, 4.2))
            plt.plot(time_plot[start_plot_idx:n_al], dist_arr[start_plot_idx:n_al], lw=1.8, color="tab:blue")
            plt.ylabel("Disturbance")
            plt.xlabel("Time (h)")
            plt.grid(True, linestyle="--", alpha=0.35)
            _savefig("disturbance.png")

    return out_dir

In [30]:
out_dir = plot_rl_results_disturbance(
    y_sp, steady_states, nFE, delta_t, time_in_sub_episodes,
    y_system, u_rl, avg_rewards, data_min, data_max, warm_start_plot,
    directory=dir_path, prefix_name="polymer_dist",
    agent=td3_agent, delta_y_storage=delta_y_storage, rewards=rewards,
    dist={"qi": qi, "qs": qs, "ha": ha}
)

In [31]:
for i in range(10):
    set_points_number = int(C_aug.shape[0])
    inputs_number = int(B_aug.shape[1])
    STATE_DIM = int(A_aug.shape[0]) + set_points_number + inputs_number
    ACTION_DIM = int(B_aug.shape[1])
    n_outputs = C_aug.shape[0]
    ACTOR_LAYER_SIZES = [512, 512, 512, 512, 512]
    CRITIC_LAYER_SIZES = [512, 512, 512, 512, 512]
    BUFFER_CAPACITY = 40000
    ACTOR_LR = 5e-5
    CRITIC_LR = 5e-4
    SMOOTHING_STD = 0.005
    NOISE_CLIP = 0.01
    # EXPLORATION_NOISE_STD = 0.01
    GAMMA = 0.995
    TAU = 0.005  # 0.01
    MAX_ACTION = 1
    POLICY_DELAY = 2
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 256
    STD_START = 0.02
    STD_END = 0.001
    STD_DECAY_RATE = 0.99992
    STD_DECAY_MODE = "exp"
    td3_agent = TD3Agent(
        state_dim=STATE_DIM,
        action_dim=ACTION_DIM,
        actor_hidden=ACTOR_LAYER_SIZES,
        critic_hidden=CRITIC_LAYER_SIZES,
        gamma=GAMMA,
        actor_lr=ACTOR_LR,
        critic_lr=CRITIC_LR,
        batch_size=BATCH_SIZE,
        policy_delay=POLICY_DELAY,
        target_policy_smoothing_noise_std=SMOOTHING_STD,
        noise_clip=NOISE_CLIP,
        max_action=MAX_ACTION,
        tau=TAU,
        std_start=STD_START,
        std_end=STD_END,
        std_decay_rate=STD_DECAY_RATE,
        std_decay_mode=STD_DECAY_MODE,
        buffer_size=BUFFER_CAPACITY,
        device=DEVICE,
        actor_freeze=ACTOR_FREEZE,
    )
    agent_path = r"C:\Users\HAMEDI\OneDrive - McMaster University\PythonProjects\Polymer_example\Data\models\agent_2507171027.pkl"
    td3_agent.load(agent_path)
    cstr = PolymerCSTR(system_params, system_design_params, system_steady_state_inputs, delta_t)
    y_system, u_rl, avg_rewards, rewards, xhatdhat, nFE, time_in_sub_episodes, y_sp, yhat, delta_y_storage, qi, qs, ha\
        = run_rl_train(cstr, y_sp_scenario, n_tests, set_points_len,
                     steady_states, min_max_dict, td3_agent, MPC_obj,
                     L, data_min, data_max, warm_start,
                     TEST_CYCLE,
                     nominal_qi, nominal_qs, nominal_hA,
                     qi_change, qs_change, ha_change,
                     reward_fn)
    out_dir = plot_rl_results_disturbance(
        y_sp, steady_states, nFE, delta_t, time_in_sub_episodes,
        y_system, u_rl, avg_rewards, data_min, data_max, warm_start_plot,
        directory=dir_path, prefix_name="polymer_dist",
        agent=td3_agent, delta_y_storage=delta_y_storage, rewards=rewards,
        dist={"qi": qi, "qs": qs, "ha": ha}
    )

Agent loaded successfully from: C:\Users\HAMEDI\OneDrive - McMaster University\PythonProjects\Polymer_example\Data\models\agent_2507171027.pkl
Sub_Episode: 1 | avg. reward: -19.90139743378253
Exploration noise: 0.018822049364146075
Sub_Episode: 2 | avg. reward: -22.829715509777124
Exploration noise: 0.017717128607266288
Sub_Episode: 3 | avg. reward: -25.671085999123015
Exploration noise: 0.016680710066604123
Sub_Episode: 4 | avg. reward: -24.36343396116448
Exploration noise: 0.01570854678273058
Sub_Episode: 5 | avg. reward: -22.154432825655167
Exploration noise: 0.014796655096667173
Sub_Episode: 6 | avg. reward: -22.673948208365086
Exploration noise: 0.013941298325942101
Sub_Episode: 7 | avg. reward: -20.388195278753468
Exploration noise: 0.013138971452688476
Sub_Episode: 8 | avg. reward: -25.506620208334642
Exploration noise: 0.01238638676104074
Sub_Episode: 9 | avg. reward: -22.15842695206463
Exploration noise: 0.011680460364975123
Sub_Episode: 10 | avg. reward: -23.489542881822675
E

C:\Users\HAMEDI\OneDrive - McMaster University\PythonProjects\Polymer_example\Simulation\system_functions.py:61: RuntimeWarning: invalid value encountered in scalar power
  CP = (2 * self.fi * kd * CI / kt) ** 0.5


Sub_Episode: 17 | avg. reward: -159.385252038862
Exploration noise: 0.007400623815872268
Sub_Episode: 18 | avg. reward: -2.6289173124812315
Exploration noise: 0.00700380176883182
Sub_Episode: 19 | avg. reward: -2.4022529030720787
Exploration noise: 0.006631581657719379
Sub_Episode: 20 | avg. reward: -2.543767170320726
Exploration noise: 0.00628243822642602
Sub_Episode: 21 | avg. reward: -2.0982674831898653
Exploration noise: 0.005954940780758777
Sub_Episode: 22 | avg. reward: -1.9830305141443518
Exploration noise: 0.005647747325847548
Sub_Episode: 23 | avg. reward: -1.957608455605889
Exploration noise: 0.005359599067017522
Sub_Episode: 24 | avg. reward: -2.1831674521477273
Exploration noise: 0.005089315251593234
Sub_Episode: 25 | avg. reward: -2.018107999907357
Exploration noise: 0.0048357883304973705
Sub_Episode: 26 | avg. reward: -2.103660669146851
Exploration noise: 0.0045979794198178735
Sub_Episode: 27 | avg. reward: -1.9466381833901603
Exploration noise: 0.004374914043746094
Sub_E